# Extract personality traits from Onboarding survey
 - Author: Mario Frei, 2024

# Imports

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import math
import pandas as pd

#import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Configurations

In [10]:
onboarding_survey_path = '../1_data/1_onboarding_survey_all_participants.csv'
weekly_survey_path = '../1_data/2_weekly_survey_data_all_participants.csv'

# Parse data

In [11]:
# Read data
df = pd.read_csv(onboarding_survey_path)

#  Rename participant ID column
df['id_participant'] = df['Experiment tag (for example orenth04, provided by the researcher)'].str.lower()
df = df.drop(columns=['Experiment tag (for example orenth04, provided by the researcher)'])

# Clean up participant ID values
df['id_participant'] = df['id_participant'].str.replace(' ', '')
df['id_participant'] = df['id_participant'].str.replace('\n', '')

df = df.set_index('id_participant', drop=True)

## Big Five personality traits
- Sources:
  - https://gosling.psy.utexas.edu/wp-content/uploads/2014/09/tipi.pdf
  - https://gosling.psy.utexas.edu/scales-weve-developed/ten-item-personality-measure-tipi/

In [12]:
# Select columns for Big Five
columns_big_five =[
    'Extraverted, enthusiastic', 
    'Critical, quarrelsome',
    'Dependable, self-disciplined', 
    'Anxious, easily upset',
    'Open to new experience, complex', 
    'Reserved, quiet',
    'Sympathetic, warm', 
    'Disorganized, careless',
    'Calm, emotionally stable', 
    'Conventional, unreactive'
]
df_5 = df[columns_big_five]

# Map responses from string scale to integer scale
map ={
    'Strongly disagree': 1,
    'Disagree': 2,
    'Somewhat disagree': 3,
    'Neither agree nor disagree': 4,
    'Somewhat agree': 5,
    'Agree': 6,
    'Strongly agree': 7
}

df_5 = df_5.replace(map)

# Compute Big Five personality traits
df_5['Extraversion'] = (df_5['Extraverted, enthusiastic'] + (8 - df_5['Reserved, quiet'])) / 2
df_5['Agreeableness'] = (df_5['Sympathetic, warm'] + (8 - df_5['Critical, quarrelsome'])) / 2
df_5['Conscientiousness'] = (df_5['Dependable, self-disciplined'] + (8 - df_5['Disorganized, careless'])) / 2
df_5['Emotional Stability'] = (df_5['Calm, emotionally stable'] + (8 - df_5['Anxious, easily upset'])) / 2
df_5['Openness to Experience'] = (df_5['Open to new experience, complex'] + (8 - df_5['Conventional, unreactive'])) / 2

# Show results
df_5.head()

C:\Users\mario\AppData\Local\Temp\ipykernel_1924\2069624797.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_5 = df_5.replace(map)


,"Extraverted, enthusiastic","Critical, quarrelsome","Dependable, self-disciplined","Anxious, easily upset","Open to new experience, complex","Reserved, quiet","Sympathetic, warm","Disorganized, careless","Calm, emotionally stable","Conventional, unreactive",Extraversion,Agreeableness,Conscientiousness,Emotional Stability,Openness to Experience
id_participant,,,,,,,,,,,,,,,
orenth01,4,2,3,3,5,5,6,5,6,7,3.5,6.0,3.0,5.5,3.0
orenth02,7,6,4,3,6,6,5,2,6,6,4.5,3.5,5.0,5.5,4.0
orenth03,2,3,3,6,4,6,6,5,3,3,2.0,5.5,3.0,2.5,4.5
orenth04,3,2,5,5,5,5,5,5,3,4,3.0,5.5,4.0,3.0,4.5
orenth05,4,3,5,5,5,4,4,5,5,4,4.0,4.5,4.0,4.0,4.5


## Highly Sensitive Person Scale (HSP-12)
- Sources:
    - https://sensitivityresearch.com/self-tests/adult-self-test/
    - https://www.sciencedirect.com/science/article/pii/S0092656623000399
    - https://psycnet.apa.org/buy/1997-05290-010
- Sensitivity: 
    - Low: 1-4
    - Medium:4-4.5
    - High: 4.5-7

In [13]:
# Select columns for Sensitivity
columns_sensitivity = [
    'Do you seem to be aware of subtleties in your environment?',
    'Are you easily overwhelmed by things like bright lights, strong smells, coarse fabrics, or sirens close by?',
    'Do you have a rich, complex inner life?',
    'Do you get rattled when you have a lot to do in a short amount of time?',
    'Are you deeply moved by the arts or music?',
    'Are you annoyed when people try to get you to do too many things at once?',
    'Do you make a point to avoid violent movies and TV shows?',
    'Do you find it unpleasant to have a lot going on at once?',
    'Do changes in your life shake you up?',
    'Do you notice and enjoy delicate or fine scents, tastes, sounds, works of art?',
    'Are you bothered by intense stimuli, like loud noises or chaotic scenes?',
    'When you must compete or be observed while performing a task, do you become so nervous or shaky that you do much worse than you would otherwise?'
    ]
df_s = df[columns_sensitivity]

# Remove strings from responses
df_s= df_s.replace({'7 (Extremely)': 7, '1 (Not at All)': 1, '1  (Not at All)': 1, '1  (Not at all)':1})
df_s.loc[:,columns_sensitivity] = df_s.loc[:,columns_sensitivity].astype(int)                                                  

# Compute sensitivity score
df_s['sensitivity']= df_s.loc[:,columns_sensitivity].mean(axis=1)

# Show result
df_s[['sensitivity']].head(10)

,sensitivity
id_participant,
orenth01,4.25
orenth02,4.666667
orenth03,4.5
orenth04,5.25
orenth05,4.333333
orenth06,3.75
orenth07,5.166667
orenth08,4.583333
orenth09,5.333333


## Satisfaction with life (SWLS)
- Sources:
    - https://labs.psychology.illinois.edu/~ediener/SWLS.html
    - https://positivepsychology.com/life-satisfaction-scales/
- Scores:
    - 30 – 35 Very high score; highly satisfied 
    - 25 - 29 High score 
    - 20 – 24 Average score 
    - 15 – 19 Slightly below average in life satisfaction 
    - 10 – 14 Dissatisfied
    -  5 -  9 Exremely dissatisfied

In [14]:
# Select relevant columns for Satisfaction with life score
columns_swls = [
    'In most ways my life is close to my ideal.',
    'The conditions of my life are excellent.',
    'I am satisfied with my life.',
    'So far I have gotten the important things I want in life.',
    'If I could live my life over, I would change almost nothing.']
df_swls = df[columns_swls]

# Convert responses from string scale to integer scale
map = {
    'Strongly disagree': 1, 
    'Disagree': 2,
    'Somewhat disagree': 3,
    'Neither agree nor disagree': 4, 
    'Somewhat agree': 5, 
    'Agree': 6, 
    'Strongly agree': 7
    }
df_swls= df_swls.replace(map)
df_swls.loc[:,columns_swls] = df_swls.loc[:,columns_swls].astype(int)

# Compute Satisfaction with life score
df_swls['swls'] = df_swls.loc[:,columns_swls].sum(axis=1)
df_swls['swls_str'] = df_swls['swls'].apply(lambda x: 'Extremely Dissatisfied' if x <= 9 else 
                                                      'Dissatisfied' if x <= 14 else 
                                                      'Slightly below average in life satisfaction' if x <= 19 else
                                                      'Average score' if x <= 24 else
                                                      'High score' if x <= 29 else
                                                      'HIghly satisfied')

# Show result
df_swls.head()

C:\Users\mario\AppData\Local\Temp\ipykernel_1924\827251819.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_swls= df_swls.replace(map)


,In most ways my life is close to my ideal.,The conditions of my life are excellent.,I am satisfied with my life.,So far I have gotten the important things I want in life.,"If I could live my life over, I would change almost nothing.",swls,swls_str
id_participant,,,,,,,
orenth01,4,5,5,3,5,22,Average score
orenth02,6,5,6,2,7,26,High score
orenth03,3,4,5,5,3,20,Average score
orenth04,3,5,4,4,2,18,Slightly below average in life satisfaction
orenth05,5,5,5,5,4,24,Average score


# Consolidate scores

In [15]:
# Consolidate all scores
df_all = pd.DataFrame(index=df.index)

df_all.loc[:, ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional Stability', 'Openness to Experience']] = df_5[['Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional Stability', 'Openness to Experience']]
df_all.loc[:,'swls'] = df_swls['swls']
#df_all.loc[:, 'swls_str'] = df_swls['swls_str']
df_all.loc[:, 'sensitivity'] = df_s['sensitivity']

df_all.to_csv('./3_all_personality_scores.csv')
df_all.head()

,Extraversion,Agreeableness,Conscientiousness,Emotional Stability,Openness to Experience,swls,sensitivity
id_participant,,,,,,,
orenth01,3.5,6.0,3.0,5.5,3.0,22,4.25
orenth02,4.5,3.5,5.0,5.5,4.0,26,4.666667
orenth03,2.0,5.5,3.0,2.5,4.5,20,4.5
orenth04,3.0,5.5,4.0,3.0,4.5,18,5.25
orenth05,4.0,4.5,4.0,4.0,4.5,24,4.333333


In [16]:
# Merge all scores with weekly survey
df_weekly = pd.read_csv(weekly_survey_path)
df_weekly['Extraversion'] = df_weekly['id_participant'].map(df_all['Extraversion'])
df_weekly['Agreeableness'] = df_weekly['id_participant'].map(df_all['Agreeableness'])
df_weekly['Conscientiousness'] = df_weekly['id_participant'].map(df_all['Conscientiousness'])
df_weekly['Emotional Stability'] = df_weekly['id_participant'].map(df_all['Emotional Stability'])
df_weekly['Openness to Experience'] = df_weekly['id_participant'].map(df_all['Openness to Experience'])
df_weekly['swls'] = df_weekly['id_participant'].map(df_all['swls'])
df_weekly['sensitivity'] = df_weekly['id_participant'].map(df_all['sensitivity'])
df_weekly.head()

df_weekly.to_csv('./4_weekly_survey_with_personality.csv', index=False)